# Logos of Most Frequenet URs of Virus Types

## Functions 

In [1]:
import pandas as pd

def get_top_URs_from_UR_rnd(xlsx_file, top_num, dtype_rnd = {'signf_sorted':str, 'tot_num_Fall_sort':int}):
    '''Returns the top top_num URs and their frequency from a UR_rnd_m<val>_<type>.xlsx file'''
    df_rnd = pd.read_excel(xlsx_file, header=0, dtype=dtype_rnd)
    columns_names = list(df_rnd.columns)
    #print('Found {} columns:'.format(len(columns_names)))
    return {df_rnd.loc[s][columns_names[0]] : df_rnd.loc[s][columns_names[1]] for s in range(top_num)}



In [2]:
import myseq_logo as mysl
import pald_funcs as mypal # my palindrome functions
import os
import numpy as np
from Bio import motifs, SeqIO
from Bio.Seq import Seq
import re
from collections import deque
from termcolor import colored
from pprint import pprint
import matplotlib.pyplot as plt

vtypes = {
    #'ssRNA' : 527,
    #'ssDNA' : 309,
    #'dsRNA' : 69,
    'dsDNA' : 1656
}
mlens = [3, 4, 5]
most_freq_num = 10
backg = {'A':0.25,'C':0.25,'G':0.25,'T':0.25}  # background for PSSM 
save_fig_file = '' #'v_URs_dsDNA_diff.png'
# ===========================================================================================
#print('Virus types:')
#for k, v in vtypes.items(): print(k, v, sep=': ')
base_file = '/Users/yoramzarai/work/school/Simulation/Viruses/Data_stats/UR_rnd_m'

fig, axs = plt.subplots(len(vtypes.keys()), len(mlens), figsize=(14,3), squeeze=False)
for i, vtype in enumerate(vtypes.keys()):
    for j, mlen in enumerate(mlens):
        xlsx_file = base_file + str(mlen) + '_' + vtype + '.xlsx'
        top_UR = get_top_URs_from_UR_rnd(xlsx_file, most_freq_num)
        eql_nt = {s for s in top_UR.keys() if len(set(s))==1}
        diff_nt = set(top_UR.keys()) - eql_nt
        gc_nt = {s for s in top_UR.keys() if re.search('[CG]{'+str(mlen)+'}',s)}
        at_nt = {s for s in top_UR.keys() if re.search('[AT]{'+str(mlen)+'}',s)}
        
        # CHOOSE here what is proc_nt (the URs to show logos)
        proc_nt = diff_nt
        #proc_nt = gc_nt
        #proc_nt = at_nt
        # generate sequence logo
        pfm, pwm, pssm, m, cons = mysl.compute_pssm(proc_nt, backg, backg)
        rel_info = mysl.calc_rel_info(m, 'no')
        maxy = mysl.gen_nt_sequence_logo(axs[i,j], rel_info)
        axs[i,j].set_title('m'+str(mlen)+' : '+vtype+' ('+str(len(proc_nt))+' cURs)', \
                          fontsize=22, color='blue')
        axs[i,j].set_ylabel('Bits')
        axs[i,j].set_xlabel('NT index')
        axs[i,j].axis([0.5, mlen+0.5, 0, maxy])
        axs[i,j].set_xticks(range(1, mlen+1))
        
        if mlen==4:
            ispald = mypal.is_palindromic(list(proc_nt))
            for seq, ispal in zip(proc_nt, ispald): print('{} is {} palindrome'.format(seq, ispal))
            pald_seq = [list(proc_nt)[i] for i in range(len(proc_nt)) if ispald[i]]
            print('Palindromes are: {}'.format(' '.join(pald_seq)))
        
plt.tight_layout()
if save_fig_file!='': 
    plt.savefig(save_fig_file, dpi=200)
    path = !pwd  # shell command
    print('Figure saved in {}/{}'.format(path[0], save_fig_file))


/Users/yoramzarai/work/school/Simulation/Viruses/jupyter/myseq_logo.py:102: RuntimeWarning: divide by zero encountered in log2
  return [Hg+sum([pwm[b][l]*np.nan_to_num(np.log2(pwm[b][l])) for b in bases])\


TCGA is True palindrome
GGCC is True palindrome
CCGG is True palindrome
AATT is True palindrome
GATC is True palindrome
CGCG is True palindrome
Palindromes are: TCGA GGCC CCGG AATT GATC CGCG
